<a href="https://colab.research.google.com/github/gupta24789/siamese-networks/blob/main/siamese_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !wget https://github.com/gupta24789/siamese-networks/raw/main/data.zip
# !unzip data.zip

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
import random
import pandas as pd
import numpy as np
import itertools
from nltk import tokenize
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import DataLoader, Dataset


import pytorch_lightning as pl

## Set Seed

In [ ]:
seed = 34
random.seed(seed)
torch.manual_seed(seed)
pl.seed_everything(seed)

Seed set to 34


34

## Read Data

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
test_df = test_df.dropna().reset_index(drop = True)

print(f'train shape : {train_df.shape}')
print(f'test shape : {test_df.shape}')

train shape : (283045, 6)
test shape : (121305, 6)


In [ ]:
train_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,27186,54210,54211,What will happen if Google starts charging for...,Is it normal to Google search every question y...,0
1,246439,485308,485309,Why are bats associated with vampires?,Do vampires get periods?,0
2,298392,586093,586094,How can I start learning data science?,How can I start learning data science and beco...,1


In [ ]:
test_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,291513,572739,572740,What Rolling Stone song has the lyrics “oh and...,What's the point in living if you're so depres...,0
1,58652,116675,76915,Do you have a Business-Plan to help the poor p...,Suppose you run a mobile business which genera...,0
2,118294,139357,234460,Where's a good university to study Computer Sc...,What are some good UK universities in computer...,1


## Prepare Data


The data is setup so that $v_{1\_1}$ and $v_{2\_1}$ represent duplicate inputs, but they are not duplicates with any other rows in the batch. This means $v_{1\_1}$ and $v_{2\_1}$ (green and green) have more similar vectors than say $v_{1\_1}$ and $v_{2\_2}$ (green and magenta).

<img src = 'images/v1v2_stacked.png' width="width" height="height" style="height:250px;"/>

In [ ]:
## Create vocab using duplicates questions only
Q1_train_sents = train_df.loc[train_df.is_duplicate==1,'question1'].tolist()
Q2_train_sents = train_df.loc[train_df.is_duplicate==1,'question2'].tolist()

In [ ]:
Q1_train_sents[:3]

['How can I start learning data science?',
 'Which is the best book to understand tensors?',
 'Who viewed my profile on Instagram?']

In [ ]:
Q2_train_sents[:3]

['How can I start learning data science and become master in it?',
 'Which is the best book to study TENSOR for general relativity from basic?',
 'Can people see if you have viewed their instagram?']

In [ ]:
## merge q1 & q2
train_sents = Q1_train_sents + Q2_train_sents
tokens = [tokenize.word_tokenize(sent) for sent in train_sents]
tokens = list(set(itertools.chain.from_iterable(tokens)))

special_tokens = ['__PAD__', '__UNK__']
tokens = special_tokens + tokens

vocab = {w:i for i,w in enumerate(tokens)}
idx2word = {i:w for w,i in vocab.items()}

UNK_ID = vocab['__UNK__']
PAD_ID = vocab['__PAD__']

print(f"Vocab {len(vocab)}")
print(f"PAD ID : {PAD_ID}")
print(f"UNK ID : {UNK_ID}")

Vocab 35289
PAD ID : 0
UNK ID : 1


In [ ]:
def encode_sent_to_number(sent):
    sent_list = tokenize.word_tokenize(sent)
    encoded_sent = []

    for w in sent_list:
        encoded_sent.append(vocab.get(w, UNK_ID))

    return encoded_sent

In [ ]:
Q1_train_encoded = [encode_sent_to_number(sent) for sent in Q1_train_sents]
Q2_train_encoded = [encode_sent_to_number(sent) for sent in Q2_train_sents]

In [ ]:
Q1_test_encoded = [encode_sent_to_number(sent) for sent in test_df.question1.tolist()]
Q2_test_encoded = [encode_sent_to_number(sent) for sent in test_df.question2.tolist()]
y_test = test_df.is_duplicate.tolist()

## Data Loader

In [ ]:
def custom_collate(batch):

    q1 = [torch.tensor(item[0]) for item in batch]
    q1_lengths = torch.tensor([len(item[0]) for item in batch] )


    q2 = [torch.tensor(item[1]) for item in batch]
    q2_lengths = torch.tensor([len(item[1]) for item in batch])

    padded_q1 = pad_sequence(q1, batch_first= True, padding_value= PAD_ID)
    padded_q2 = pad_sequence(q2, batch_first= True, padding_value= PAD_ID)

    batch = {"q1": padded_q1, "q2": padded_q2,"q1_lengths": q1_lengths, "q2_lengths": q2_lengths}
    return batch

In [ ]:
batch_size = 3
train_dl = DataLoader(list(zip(Q1_train_encoded,Q2_train_encoded)), batch_size = batch_size, shuffle=False, collate_fn= custom_collate)
example = next(iter(train_dl))
example['q1'].shape, example['q2'].shape, example['q1_lengths'].shape, example['q2_lengths'].shape

(torch.Size([3, 9]), torch.Size([3, 14]), torch.Size([3]), torch.Size([3]))

In [ ]:
example['q1']

tensor([[10099, 28674, 10107, 28501,  6615,  9219, 28539, 25610,     0],
        [21715, 21347, 32108,  4871, 24280,  1962, 11812,  6551, 25610],
        [18090, 28904, 12971, 25354,  3562, 12178, 25610,     0,     0]])

In [ ]:
example['q1_lengths']

tensor([8, 9, 7])

In [ ]:
## dataloaders
batch_size = 256
train_dl = DataLoader(list(zip(Q1_train_encoded,Q2_train_encoded)), batch_size = batch_size, shuffle=True, collate_fn= custom_collate)
val_dl = DataLoader(list(zip(Q1_test_encoded,Q2_test_encoded)), batch_size = batch_size, shuffle=False, collate_fn= custom_collate)

## Model

You will now implement the `TripletLoss`.<br>
As explained in the lecture, loss is composed of two terms. One term utilizes the mean of all the non duplicates, the second utilizes the *closest negative*. Our loss expression is then:

\begin{align}
 \mathcal{Loss_1(A,P,N)} &=\max \left( -cos(A,P)  + mean_{neg} +\alpha, 0\right) \\
 \mathcal{Loss_2(A,P,N)} &=\max \left( -cos(A,P)  + closest_{neg} +\alpha, 0\right) \\
\mathcal{Loss(A,P,N)} &= mean(Loss_1 + Loss_2) \\
\end{align}

In [ ]:
class SiameseModel(pl.LightningModule):

    def __init__(self, vocab_size, emb_dim, hidden_dim, margin, threshold, learning_rate, bidirectional, dropout, num_layers):
        super().__init__()
        self.bidirectional = bidirectional
        self.learning_rate = learning_rate
        self.margin = margin
        self.threhold = threshold
        self.val_loss = []

        # layers
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD_ID)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first = True, num_layers = num_layers, bidirectional = bidirectional, dropout = dropout)

    def loss_fn(self, scores):

        device = "cuda" if scores.is_cuda else "cpu"
        batch_size = len(scores)
        positive = torch.diagonal(scores)
        negative_zero_on_duplicate = scores * (1.0 - torch.eye(batch_size)).to(device)
        mean_negative = torch.sum(negative_zero_on_duplicate, axis=1) / (batch_size-1)

        negative_without_positive = scores - 2.0 * torch.eye(batch_size).to(device)
        closest_negative, _ = negative_without_positive.max(axis=1)

        # print(f"scores : \n{scores}")
        # print("positive : ", positive)
        # print(f"neg : {negative_zero_on_duplicate}")
        # print("mean_negative : ", mean_negative)
        # print("closest_negative : ", closest_negative)

        triplet_loss1 = torch.maximum(torch.tensor(0.0), self.margin - positive + closest_negative)
        triplet_loss2 = torch.maximum(torch.tensor(0.0), self.margin - positive + mean_negative)
        triplet_loss = torch.mean(triplet_loss1 + triplet_loss2)
        return triplet_loss

    def get_normalize_vector(self, q, q_lengths):
        out = self.embedding(q)
        ## out : [batch size, sent len, emb dim]
        packed_input = pack_padded_sequence(out, lengths= q_lengths.to('cpu'), batch_first = True, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_input)
        out, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first = True)
        ## out : [batch size, sent len, hidden dim * bidirectional]
        # hidden : [num_layers * bidirectional, batch size,hidden dim]

        # out = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        ## out : [batch size, hidden_dim * bidirectional]
        out = torch.mean(out, dim = 1)
        ## [batch size, hidden dim * bidirectional]
        out = F.normalize(out, p = 2)

        return out

    def forward(self, q1, q2, q1_lengths, q2_lengths):
        q1_vec = self.get_normalize_vector(q1, q1_lengths)
        # print(q1_vec.shape)
        q2_vec = self.get_normalize_vector(q2, q2_lengths)
        # print(q2_vec.shape)
        return (q1_vec, q2_vec)

    def training_step(self, batch):
        q1,q2,q1_lengths, q2_lengths = batch['q1'], batch['q2'],batch['q1_lengths'], batch['q2_lengths']
        q1_vec,q2_vec = self(q1,q2,q1_lengths, q2_lengths)
        scores = torch.matmul(q1_vec, q2_vec.T)
        loss = self.loss_fn(scores)
        self.log_dict({"train_loss": loss}, on_step = False, on_epoch = True, prog_bar=True)
        return loss

    def validation_step(self, batch):
        q1,q2,q1_lengths, q2_lengths = batch['q1'], batch['q2'],batch['q1_lengths'], batch['q2_lengths']
        q1_vec,q2_vec = self(q1,q2,q1_lengths, q2_lengths)
        scores = torch.matmul(q1_vec, q2_vec.T)
        loss = self.loss_fn(scores)
        self.val_loss.append(loss.item())
        self.log_dict({"val_loss": loss}, on_step = False, on_epoch = True, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):
        print(f"Current Epoch : {self.current_epoch} Valiadtion Loss : {np.mean(self.val_loss)}")
        self.val_loss = []

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.learning_rate)
        return optimizer

In [ ]:
## test model architecture
model = SiameseModel(len(vocab), emb_dim = 100, hidden_dim = 150, margin = 0.25, threshold = 0.7, learning_rate = 1e-3, bidirectional= True, num_layers = 2, dropout = 0.25)
print(model)
q1, q2, q1_lengths, q2_lengths = example['q1'], example['q2'],example['q1_lengths'],example['q2_lengths']
v1,v2 = model(q1, q2, q1_lengths, q2_lengths)
scores = torch.matmul(v1,v2.T)
loss = model.loss_fn(scores)
print(loss)
print(scores)

SiameseModel(
  (embedding): Embedding(35289, 100, padding_idx=0)
  (lstm): LSTM(100, 150, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)
)
tensor(0.3421, grad_fn=<MeanBackward0>)
tensor([[0.8784, 0.6379, 0.6515],
        [0.7014, 0.7593, 0.6195],
        [0.7021, 0.6844, 0.6259]], grad_fn=<MmBackward0>)


In [ ]:
## Model Training
model= SiameseModel(len(vocab), emb_dim = 100, hidden_dim = 150, margin = 0.25, threshold = 0.7, learning_rate = 1e-3, bidirectional= True, num_layers = 2, dropout = 0.1)

callbacks = pl.callbacks.ModelCheckpoint(dirpath = "checkpoints_logs",
                                         filename = '{epoch}-{val_loss:.2f}',
                                          mode = "min",
                                          monitor = "val_loss",
                                          save_last = True,
                                          save_top_k=-1)


trainer = pl.Trainer(accelerator= "gpu",
           max_epochs=10,
           check_val_every_n_epoch = 1,
           callbacks = [callbacks])

trainer.fit(model, train_dl, val_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 3.5 M 
1 | lstm      | LSTM      | 844 K 
----------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.495    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Current Epoch : 0 Valiadtion Loss : 0.39718714356422424


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Current Epoch : 8 Valiadtion Loss : 0.20134402858682826


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Current Epoch : 9 Valiadtion Loss : 0.2016061165287525


## Test the accuracy

In [ ]:
model = model.eval()

total = 0
correct = 0
total = 0
threshold = 0.7
device = "cuda"

model.to(device)

for batch in tqdm_notebook(val_dl):
    q1,q2, q1_lengths, q2_lengths = batch['q1'], batch['q2'],batch['q1_lengths'], batch['q2_lengths']
    batch_size = q1.shape[0]

    q1 = q1.to(device)
    q2 = q2.to(device)
    v1,v2 = model(q1,q2,q1_lengths, q2_lengths)
    v1.to(device)
    v2.to(device)
    scores = torch.matmul(v1, v2.T)
    res = torch.diag(scores)
    y_pred = (res>threshold).long()
    y_true = torch.tensor(y_test[total: total + batch_size])

    correct += sum(y_true == y_pred.cpu()).item()
    total += len(y_pred)

print(f"Test Accuracy : {correct/total}")

/tmp/ipykernel_3694/4028790655.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(val_dl):


  0%|          | 0/474 [00:00<?, ?it/s]

Test Accuracy : 0.7192448786117638


## Predict

In [ ]:
model = model.eval()
model.to("cpu")

def predict(q1, q2, threshold = 0.7):

    q1_words = tokenize.word_tokenize(q1)
    q2_words = tokenize.word_tokenize(q2)

    q1_encoded = [vocab.get(w, UNK_ID) for w in q1_words]
    q2_encoded = [vocab.get(w, UNK_ID) for w in q2_words]

    q1_len = len(q1_encoded)
    q2_len = len(q2_encoded)

    q1 = torch.tensor(q1_encoded).view(1,-1)
    q2 = torch.tensor(q2_encoded).view(1,-1)

    q1_lengths = torch.tensor([q1.shape[1]], dtype = torch.long)
    q2_lengths = torch.tensor([q2.shape[1]], dtype = torch.long)

    v1, v2 = model(q1,q2, q1_lengths, q2_lengths)
    score = torch.matmul(v1, v2.T)
    if score > threshold:
        return ("Duplicated", score.item())
    else:
        return ("Not Duplicated", score.item())

In [ ]:
question1 = "When will I see you?"
question2 = "When can I see you again?"
predict(question1 , question2)

('Duplicated', 0.7735307216644287)

In [ ]:
question1 = "Do they enjoy eating the dessert?"
question2 = "Do they like hiking in the desert?"
predict(question1 , question2)

('Not Duplicated', 0.14368796348571777)